In [5]:
import os
import sys
import multiprocessing as mp

# a hack to import module from different directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tqdm
import numpy as np
import commpy as cp
from commpy.modulation import QAMModem
from commpy.channelcoding import Trellis

from radioml.radio_receivers import ModularReceiver, End2EndReceiver
from radioml.decoders import ViterbiDecoder, NeuralDecoder
from radioml.demodulators import ClassicDemodulator, NeuralDemodulator
import radioml.utils as utils

## Define paramters for the comparison

In [6]:
class Params:
    SNR_RANGE = [0.0, 5.0, 10.0, 15.0, 16.0, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0]
    NUM_SAMPLES = 100
    BLOCK_LENGTH = 100
    BPSK, QPSK, QAM16, QAM64 = 2, 4, 16, 64
    
# For encoding/decoding convolutional codes
G = np.array([[0o7, 0o5]]) 
M = np.array([2])
trellis = Trellis(M, G, feedback=0o7, code_type='rsc')
modem = QAMModem(m=Params.QPSK)

## Define helper function to simulate sending signals over AWGN Channel

In [7]:
def generate_signal_over_awgn(modem, block_length=100, snr_dB=15.0):
    message_bits   = np.random.randint(0, 2, block_length)
    encoded_bits   = cp.channelcoding.conv_encode(message_bits, trellis)
    modulated_bits = modem.modulate(encoded_bits)
    corrupted_bits = cp.channels.awgn(modulated_bits, snr_dB, rate=1/2)
    
    return message_bits, modulated_bits, corrupted_bits

def signal_generator(modulation_scheme, num_examples, block_length, snr_dB):
    with mp.Pool(mp.cpu_count()) as pool:
        result = pool.starmap(generate_signal_over_awgn,
                iterable=[(modulation_scheme, block_length, snr_dB) \
                          for i in range(num_examples)])
        orignal_msg_bits, moded_bits, noisy_outputs = zip(*result)
    return (np.array(orignal_msg_bits), np.array(moded_bits), np.array(noisy_outputs))

## Define two Receivers:
 
* **Baseline Receiver**: Classic Demod + Viterbi
* **End2End Receiver**: an RNN

In [8]:
classic_demod   = ClassicDemodulator(modem)
viterbi_decoder = ViterbiDecoder(trellis, tb_depth=15, decoding_type='hard')

baseline_receiver = ModularReceiver(classic_demod, viterbi_decoder)
end2end_receiver = End2EndReceiver('test')

## Evaluate Baseline  on multiple SNRs

* For each SNR, generate `Params.NUM_SAMPLES` message bits and noisy signals.
* Run Baseline/Modular version on the same inputs
* Compute Bit error Rate, Block error rate

In [ ]:
ber_logs, bler_logs = [], []
for i, snr in enumerate(Params.SNR_RANGE):
    (original_bits, clean_signals, noisy_signals) = signal_generator(
        modem,
        Params.NUM_SAMPLES, 
        Params.BLOCK_LENGTH, 
        snr)
    
    # ####################################
    # Run Baseline/Neral Receiver
    # ####################################
    with mp.Pool(mp.cpu_count()) as pool:
        baseline_estimated = pool.map(baseline_receiver, [i for i in noisy_signals])
  
    nn_estimated = neural_receiver(noisy_signals, True)
    
    # ####################################
    # Measure Accuracy / error
    # #################################### 
    # For decoder
    ber, bler        = utils.get_ber_bler(np.array(baseline_estimated)[:, :100], 
                                          original_bits)
    nn_ber, nn_bler  = utils.get_ber_bler(nn_estimated, original_bits)
    ber_logs.append([ber, nn_ber])
    bler_logs.append([bler, nn_bler])
    
    print('SNR_dB = %f' % snr)
    print('\t[Modular]  Ber = {:.4f} | Bler ={:.4f}'.format(nn_ber, nn_bler))
    print('\t[Baseline] Ber = {:.4f} | Bler ={:.4f} '.format(ber, bler))

### Compare Decoder Performance 

In [ ]:
import matplotlib.pyplot as plt

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
utils.visualize_ber_bler(ax1, ax2, ber_logs, bler_logs, Params.SNR_RANGE)
ax1.semilogy()
ax2.semilogy()